#ADSMINITRATIVE GEOCODING

##Import Dependencies

In [ ]:
!pip install pandas requests openpyxl tqdm

import pandas as pd
import requests
from tqdm.notebook import tqdm
from google.colab import files

##Upload File

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_excel(filename)

##Data Cleaning

In [ ]:
# Clean up column names
df.columns = df.columns.str.strip()

##Processing Using GMAPS Geocoding API

In [ ]:
API_KEY = "API KEY"

#Function to get administratif Area
def get_administrative_info(address):
    try:
        url = f"https://maps.googleapis.com/maps/api/geocode/json?address={requests.utils.quote(address)}&key={API_KEY}"
        response = requests.get(url)
        data = response.json()
        if data["status"] != "OK":
            return None, None, None, None
        comps = data["results"][0]["address_components"]

        province, kabupaten, kecamatan, kelurahan = None, None, None, None
        for comp in comps:
            if "administrative_area_level_1" in comp["types"]:
                province = comp["long_name"]
            elif "administrative_area_level_2" in comp["types"]:
                kabupaten = comp["long_name"]
            elif "administrative_area_level_3" in comp["types"]:
                kecamatan = comp["long_name"]
            elif "administrative_area_level_4" in comp["types"]:
                kelurahan = comp["long_name"]

        return province, kabupaten, kecamatan, kelurahan

    except Exception as e:
        return None, None, None, None

provinces, kabupatens, kecamatans, kelurahans = [], [], [], []

for addr in tqdm(df["- Alamat Store"], desc="Mendeteksi wilayah administratif"):
    if pd.isna(addr):
        provinces.append(None)
        kabupatens.append(None)
        kecamatans.append(None)
        kelurahans.append(None)
        continue
    prov, kab, kec, kel = get_administrative_info(addr)
    provinces.append(prov)
    kabupatens.append(kab)
    kecamatans.append(kec)
    kelurahans.append(kel)

df["Provinsi"] = provinces
df["Kabupaten/Kota"] = kabupatens
df["Kecamatan"] = kecamatans
df["Kelurahan/Desa"] = kelurahans



##Export Result

In [ ]:
output_name = "result_geocode.xlsx"
df.to_excel(output_name, index=False)
files.download(output_name)